Creating Bullpen Features

    - Bullpen = relief pitchers (i.e. a pitcher who enters the game who wasn't the starting pitcher [[INCLUDES CLOSER]])
    - Bullpen dynamics are very complicated! For simplicity, we will just consider the 'team bullpen' rather than individual players
    - For each game, we can look at the performance of the bullpen by subtracting the SP from the pitching stats.
    - Then, do 'n' game lookbacks (similar to how we did team hitting data) to create features based on recent bullpen performance
    - This will not, however, account for which pitchers are rested/available for the game in question

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

pd.set_option('display.max_columns',5000)
pd.set_option('display.max_rows',5000)

In [2]:
df = pd.read_csv('df_bp5.csv', low_memory=False)
df.shape

(116046, 451)


Plan of Attack

    For each game, we have summary stats of the starting pitchers performance
    Also have (or can calculate) those stats for the overall game
    Difference between those two will be accountable to the bullpen

In [3]:
df.sample(10)

,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,runs_h,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore_v,linescore_h,AB_v,H_v,2B_v,3B_v,HR_v,RBI_v,SH_v,SF_v,HBP_v,BB_v,IBB_v,SO_v,SB_v,CS_v,GIDP_v,CI_v,LOB_v,P_num_v,ERind_v,ERteam_v,WP_v,balk_v,PO_v,ASST_v,ERR_v,PB_v,DP_v,TP_v,AB_h,H_h,2B_h,3B_h,HR_h,RBI_h,SH_h,SF_h,HBP_h,BB_h,IBB_h,SO_h,SB_h,CS_h,GIDP_h,CI_h,LOB_h,P_num_h,ERind_h,ERteam_h,WP_h,balk_h,PO_h,ASST_h,ERR_h,PB_h,DP_h,TP_h,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id_v,mgr_name_v,mgr_id_h,mgr_name_h,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id_v,pitcher_start_name_v,pitcher_start_id_h,pitcher_start_name_h,batter1_name_v,batter1_id_v,batter1_pos_v,batter2_name_v,batter2_id_v,batter2_pos_v,batter3_name_v,batter3_id_v,batter3_pos_v,batter4_name_v,batter4_id_v,batter4_pos_v,batter5_name_v,batter5_id_v,batter5_pos_v,batter6_name_v,batter6_id_v,batter6_pos_v,batter7_name_v,batter7_id_v,batter7_pos_v,batter8_name_v,batter8_id_v,batter8_pos_v,batter9_name_v,batter9_id_v,batter9_pos_v,batter1_name_h,batter1_id_h,batter1_pos_h,batter2_name_h,batter2_id_h,batter2_pos_h,batter3_name_h,batter3_id_h,batter3_pos_h,batter4_name_h,batter4_id_h,batter4_pos_h,batter5_name_h,batter5_id_h,batter5_pos_h,batter6_name_h,batter6_id_h,batter6_pos_h,batter7_name_h,batter7_id_h,batter7_pos_h,batter8_name_h,batter8_id_h,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info,season,run_diff,home_victory,run_total,date_dblhead,BATAVG_162_h,BATAVG_162_v,OBP_162_h,OBP_162_v,SLG_162_h,SLG_162_v,OBS_162_h,OBS_162_v,SB_162_h,SB_162_v,CS_162_h,CS_162_v,ERR_162_h,ERR_162_v,BATAVG_30_h,BATAVG_30_v,OBP_30_h,OBP_30_v,SLG_30_h,SLG_30_v,OBS_30_h,OBS_30_v,SB_30_h,SB_30_v,CS_30_h,CS_30_v,ERR_30_h,ERR_30_v,implied_prob_h,implied_prob_v,implied_prob_h_mid,over_under_line,over_under_result,Strt_GS_h,Strt_GS_v,Strt_IP_h,Strt_IP_v,Strt_H_h,Strt_H_v,Strt_BFP_h,Strt_BFP_v,Strt_HR_h,Strt_HR_v,Strt_R_h,Strt_R_v,Strt_ER_h,Strt_ER_v,Strt_BB_h,Strt_BB_v,Strt_IB_h,Strt_IB_v,Strt_SO_h,Strt_SO_v,Strt_SH_h,Strt_SH_v,Strt_SF_h,Strt_SF_v,Strt_WP_h,Strt_WP_v,Strt_HBP_h,Strt_HBP_v,Strt_BK_h,Strt_BK_v,Strt_2B_h,Strt_2B_v,Strt_3B_h,Strt_3B_v,Strt_IP_real_h,Strt_IP_real_v,Strt_rollsum_IP_real_10_h,Strt_rollsum_IP_real_10_v,Strt_rollsum_H_10_h,Strt_rollsum_H_10_v,Strt_rollsum_BFP_10_h,Strt_rollsum_BFP_10_v,Strt_rollsum_HR_10_h,Strt_rollsum_HR_10_v,Strt_rollsum_R_10_h,Strt_rollsum_R_10_v,Strt_rollsum_ER_10_h,Strt_rollsum_ER_10_v,Strt_rollsum_BB_10_h,Strt_rollsum_BB_10_v,Strt_rollsum_IB_10_h,Strt_rollsum_IB_10_v,Strt_rollsum_SO_10_h,Strt_rollsum_SO_10_v,Strt_rollsum_SH_10_h,Strt_rollsum_SH_10_v,Strt_rollsum_SF_10_h,Strt_rollsum_SF_10_v,Strt_rollsum_WP_10_h,Strt_rollsum_WP_10_v,Strt_rollsum_HBP_10_h,Strt_rollsum_HBP_10_v,Strt_rollsum_BK_10_h,Strt_rollsum_BK_10_v,Strt_rollsum_2B_10_h,Strt_rollsum_2B_10_v,Strt_rollsum_3B_10_h,Strt_rollsum_3B_10_v,Strt_rollsum_IP_real_35_h,Strt_rollsum_IP_real_35_v,Strt_rollsum_H_35_h,Strt_rollsum_H_35_v,Strt_rollsum_BFP_35_h,Strt_rollsum_BFP_35_v,Strt_rollsum_HR_35_h,Strt_rollsum_HR_35_v,Strt_rollsum_R_35_h,Strt_rollsum_R_35_v,Strt_rollsum_ER_35_h,Strt_rollsum_ER_35_v,Strt_rollsum_BB_35_h,Strt_rollsum_BB_35_v,Strt_rollsum_IB_35_h,Strt_rollsum_IB_35_v,Strt_rollsum_SO_35_h,Strt_rollsum_SO_35_v,Strt_rollsum_SH_35_h,Strt_rollsum_SH_35_v,Strt_rollsum_SF_35_h,Strt_rollsum_SF_35_v,Strt_rollsum_WP_35_h,Strt_rollsum_WP_35_v,Strt_rollsum_HBP_35_h,Strt_rollsum_HBP_35_v,Strt_rollsum_BK_35_h,Strt_rollsum_BK_35_v,Strt_rollsum_2B_35_h,Strt_rollsum_2B_35_v,Strt_rollsum_3B_35_h,Strt_rollsum_3B_35_v,Strt_rollsum_IP_real_75_h,Strt_rollsum_IP_real_75_v,Strt_rollsum_H_75_h,Strt_rollsum_H_75_v,Strt_rollsum_BFP_75_h,Strt_rollsum_BFP_75_v,Strt_rollsum_HR_75_h,Strt_rollsum_HR_75_v,Strt_rol

In [4]:
## Calculate some game level stats, specifically about
## relative stats for starting pitcher vs bullpen

full_innings = (df.outs_total//6)
additional_outs = df.outs_total - 6*full_innings
added_innings_pitched_h = np.minimum(additional_outs,3)/3
added_innings_pitched_v = (np.maximum(additional_outs,3)-3)/3
df['innings_pitched_h'] = full_innings + added_innings_pitched_h
df['innings_pitched_v'] = full_innings + added_innings_pitched_v
df['Bpen_IP_h'] = df['innings_pitched_h']-df['Strt_IP_real_h']
df['Bpen_IP_v'] = df['innings_pitched_v']-df['Strt_IP_real_v']
df['Bpen_BFP_h'] = df['AB_v']+df['BB_v']+df['HBP_v']-df['Strt_BFP_h']
df['Bpen_BFP_v'] = df['AB_h']+df['BB_h']+df['HBP_h']-df['Strt_BFP_v']
df['Bpen_R_h'] = df['runs_v']-df['Strt_R_h']
df['Bpen_R_v'] = df['runs_h']-df['Strt_R_v']
df['Bpen_H_h'] = df['H_v']-df['Strt_H_h']
df['Bpen_H_v'] = df['H_h']-df['Strt_H_v']
df['Bpen_HR_h'] = df['HR_v']-df['Strt_HR_h']
df['Bpen_HR_v'] = df['HR_h']-df['Strt_HR_v']
df['Bpen_2B_h'] = df['2B_v']-df['Strt_2B_h']
df['Bpen_2B_v'] = df['2B_h']-df['Strt_2B_v']
df['Bpen_3B_h'] = df['3B_v']-df['Strt_3B_h']
df['Bpen_3B_v'] = df['3B_h']-df['Strt_3B_v']
df['Bpen_BB_h'] = df['BB_v']-df['Strt_BB_h']
df['Bpen_BB_v'] = df['BB_h']-df['Strt_BB_v']
df['Bpen_HBP_h'] = df['HBP_v']-df['Strt_HBP_h']
df['Bpen_HBP_v'] = df['HBP_h']-df['Strt_HBP_v']
df['Bpen_SO_h'] = df['SO_v']-df['Strt_SO_h']
df['Bpen_SO_v'] = df['SO_h']-df['Strt_SO_v']

In [5]:
def roll_column(df, col, winsize):
    # do the standard Pandas rolling calc
    t_col = df[col].rolling(winsize, closed='left').sum().to_numpy()
    
    # for the early columns, just do a rolling sum from the beginning
    t_col[:winsize] = np.concatenate(([0],df[col].iloc[:(winsize)].cumsum().to_numpy()[:-1]))

    return(t_col)

def strip_suffix(x, suff):
    if x.endswith(suff):
        return(x[:-len(suff)])
    else:
        return(x)

In [6]:
## Process df again to get bullpen stats
def get_bullpen_team_df(team):
    visit_cols = [col for col in df.columns if not col.endswith('_h')]
    visit_cols_stripped = [strip_suffix(col,'_v') for col in visit_cols]
    home_cols = [col for col in df.columns if not col.endswith('_v')]
    home_cols_stripped = [strip_suffix(col,'_h') for col in home_cols]    
    df_team_v = df[(df.team_v==team)]
    opponent = df_team_v['team_h']
    df_team_v = df_team_v[visit_cols]
    df_team_v.columns = visit_cols_stripped
    df_team_v['home_game'] = 0
    df_team_v['opponent'] = opponent

    df_team_h = df[(df.team_h==team)]
    opponent = df_team_h['team_v']
    df_team_h = df_team_h[home_cols]
    df_team_h.columns = home_cols_stripped
    df_team_h['home_game'] = 1
    df_team_h['opponent'] = opponent

    df_team = pd.concat((df_team_h, df_team_v))
    df_team.sort_values(['date_dblhead'],inplace=True)

    er_per_ip_def = (5/9)
    h_bb_per_ip_def = 1.5
    h_bb_per_bf_def = .37
    so_per_bf_def = .2
    ip_per_game_def = 2
    bf_per_game_def = 6
    tb_bb_perc_def = .45

    cols_to_agg = ['IP', 'H','BFP', 'HR', 'R',  'BB', 'SO',  'HBP',
       '2B', '3B']
    winsizes = [10,35,75]
    for winsize in winsizes:
        for raw_col in cols_to_agg:
            col_agg = 'Bpen_'+raw_col
            new_colname = 'Bpen_rollsum_'+raw_col+'_'+str(winsize)        
            df_team[new_colname] = roll_column(df_team, col_agg, winsize)

        hit_col = 'Bpen_rollsum_H_'+str(winsize)
        bb_col = 'Bpen_rollsum_BB_'+str(winsize)
        h_bb_col = 'Bpen_H_BB_roll_'+str(winsize)
        double_col = 'Bpen_rollsum_2B_'+str(winsize)
        triple_col = 'Bpen_rollsum_3B_'+str(winsize)
        hr_col = 'Bpen_rollsum_HR_'+str(winsize)
        xb_col = 'Bpen_XB_roll_'+str(winsize)
        tb_col = 'Bpen_TB_roll_'+str(winsize)
        so_col = 'Bpen_rollsum_SO_'+str(winsize)
        so_mod_col = 'Bpen_SO_mod_'+str(winsize)
        ip_col = 'Bpen_rollsum_IP_'+str(winsize)
        ip_mod_col = 'Bpen_IP_mod_'+str(winsize)
        bf_col = 'Bpen_rollsum_BFP_'+str(winsize)
        bf_mod_col = 'Bpen_BF_mod_'+str(winsize)
        whip_col = 'Bpen_WHIP_'+str(winsize)
        so_perc_col = 'Bpen_SO_perc_'+str(winsize)
        h_bb_perc_col = 'Bpen_H_BB_perc_'+str(winsize)
        h_bb_mod_col = 'Bpen_H_BB_mod_'+str(winsize)
        h_bb_mod2_col = 'Bpen_Bpen_H_BB_mod2_'+str(winsize)
        tb_bb_mod_col = 'Bpen_TB_BB_mod_'+str(winsize)
        tb_bb_perc_col = 'Bpen_TB_BB_perc_'+str(winsize)
        df_team[h_bb_col] = df_team[hit_col]+df_team[bb_col]
        df_team[xb_col] = df_team[double_col]+2*df_team[triple_col]+3*df_team[hr_col]
        df_team[tb_col] = df_team[hit_col]+df_team[xb_col]
        df_team[ip_mod_col] = np.maximum(df_team[ip_col], winsize*ip_per_game_def)
        df_team[bf_mod_col] = np.maximum(df_team[bf_col], winsize*bf_per_game_def)
        df_team[h_bb_mod_col] = df_team[h_bb_col] + h_bb_per_ip_def*(df_team[ip_mod_col]-df_team[ip_col])
        df_team[h_bb_mod2_col] = df_team[h_bb_col] + h_bb_per_bf_def*(df_team[bf_mod_col]-df_team[bf_col])
        df_team[so_mod_col] = df_team[so_col] + so_per_bf_def*(df_team[bf_mod_col]-df_team[bf_col])
        df_team[tb_bb_mod_col] = (df_team[tb_col] + df_team[bb_col])+ tb_bb_perc_def*(df_team[bf_mod_col]-df_team[bf_col])
        df_team[whip_col] = df_team[h_bb_mod_col]/df_team[ip_mod_col]
        df_team[so_perc_col] = df_team[so_mod_col]/df_team[bf_mod_col]
        df_team[tb_bb_perc_col] = df_team[tb_bb_mod_col]/df_team[bf_mod_col]
        df_team[h_bb_perc_col] = df_team[h_bb_mod2_col]/df_team[bf_mod_col]
       
    df_team.set_index('date_dblhead', inplace=True)
    return(df_team)

In [8]:
bullpen_team_data_dict = {}
for team in df.team_v.unique():
    bullpen_team_data_dict[team] = get_bullpen_team_df(team)

In [9]:
np.array(bullpen_team_data_dict['CIN'].columns)

array(['date', 'dblheader_code', 'day_of_week', 'team', 'league',
       'game_no', 'runs', 'outs_total', 'day_night', 'completion_info',
       'forfeit_info', 'protest_info', 'ballpark_id', 'attendance',
       'game_minutes', 'linescore', 'AB', 'H', '2B', '3B', 'HR', 'RBI',
       'SH', 'SF', 'HBP', 'BB', 'IBB', 'SO', 'SB', 'CS', 'GIDP', 'CI',
       'LOB', 'P_num', 'ERind', 'ERteam', 'WP', 'balk', 'PO', 'ASST',
       'ERR', 'PB', 'DP', 'TP', 'ump_HB_id', 'ump_HB_name', 'ump_1B_id',
       'ump_1B_name', 'ump_2B_id', 'ump_2B_name', 'ump_3B_id',
       'ump_3B_name', 'ump_LF_id', 'ump_LF_name', 'ump_RF_id',
       'ump_RF_name', 'mgr_id', 'mgr_name', 'pitcher_id_w',
       'pitcher_name_w', 'pitcher_id_l', 'pitcher_name_l', 'pitcher_id_s',
       'pitcher_name_s', 'GWRBI_id', 'GWRBI_name', 'pitcher_start_id',
       'pitcher_start_name', 'batter1_name', 'batter1_id', 'batter1_pos',
       'batter2_name', 'batter2_id', 'batter2_pos', 'batter3_name',
       'batter3_id', 'batter3_pos'

In [10]:
raw_cols_to_add = ['Bpen_IP', 'Bpen_BFP', 'Bpen_R', 'Bpen_H', 'Bpen_HR', 'Bpen_2B',
       'Bpen_3B', 'Bpen_BB', 'Bpen_HBP', 'Bpen_SO',  'Bpen_rollsum_IP_10', 'Bpen_rollsum_H_10',
       'Bpen_rollsum_BFP_10', 'Bpen_rollsum_HR_10', 'Bpen_rollsum_R_10',
       'Bpen_rollsum_BB_10', 'Bpen_rollsum_SO_10', 'Bpen_rollsum_HBP_10',
       'Bpen_rollsum_2B_10', 'Bpen_rollsum_3B_10', 'Bpen_H_BB_roll_10',
       'Bpen_XB_roll_10', 'Bpen_TB_roll_10', 'Bpen_IP_mod_10',
       'Bpen_BF_mod_10', 'Bpen_H_BB_mod_10', 'Bpen_Bpen_H_BB_mod2_10',
       'Bpen_SO_mod_10', 'Bpen_TB_BB_mod_10', 'Bpen_WHIP_10',
       'Bpen_SO_perc_10', 'Bpen_TB_BB_perc_10', 'Bpen_H_BB_perc_10',
       'Bpen_rollsum_IP_35', 'Bpen_rollsum_H_35', 'Bpen_rollsum_BFP_35',
       'Bpen_rollsum_HR_35', 'Bpen_rollsum_R_35', 'Bpen_rollsum_BB_35',
       'Bpen_rollsum_SO_35', 'Bpen_rollsum_HBP_35', 'Bpen_rollsum_2B_35',
       'Bpen_rollsum_3B_35', 'Bpen_H_BB_roll_35', 'Bpen_XB_roll_35',
       'Bpen_TB_roll_35', 'Bpen_IP_mod_35', 'Bpen_BF_mod_35',
       'Bpen_H_BB_mod_35', 'Bpen_Bpen_H_BB_mod2_35', 'Bpen_SO_mod_35',
       'Bpen_TB_BB_mod_35', 'Bpen_WHIP_35', 'Bpen_SO_perc_35',
       'Bpen_TB_BB_perc_35', 'Bpen_H_BB_perc_35', 'Bpen_rollsum_IP_75',
       'Bpen_rollsum_H_75', 'Bpen_rollsum_BFP_75', 'Bpen_rollsum_HR_75',
       'Bpen_rollsum_R_75', 'Bpen_rollsum_BB_75', 'Bpen_rollsum_SO_75',
       'Bpen_rollsum_HBP_75', 'Bpen_rollsum_2B_75', 'Bpen_rollsum_3B_75',
       'Bpen_H_BB_roll_75', 'Bpen_XB_roll_75', 'Bpen_TB_roll_75',
       'Bpen_IP_mod_75', 'Bpen_BF_mod_75', 'Bpen_H_BB_mod_75',
       'Bpen_Bpen_H_BB_mod2_75', 'Bpen_SO_mod_75', 'Bpen_TB_BB_mod_75',
       'Bpen_WHIP_75', 'Bpen_SO_perc_75', 'Bpen_TB_BB_perc_75',
       'Bpen_H_BB_perc_75']

cols_to_add = [col+suff for col in raw_cols_to_add for suff in ['_h','_v']]
col_add_dict = {col:np.zeros(df.shape[0]) for col in cols_to_add}

In [11]:
for i in range(df.shape[0]):
    row = df.iloc[i,:]
    if i%1000==0:
        print(i)
    try:
        home_team = row['team_h']
        visit_team = row['team_v']
        date_dblhead = row['date_dblhead']
        curr_df = bullpen_team_data_dict[home_team]
    except: (KeyError)
    pass
    if date_dblhead in curr_df.index:
        for col in raw_cols_to_add:
            col_add_dict[col+'_h'][i] = curr_df.loc[date_dblhead,col]
    else:
            print(f'no match for {home_team} date {date_dblhead}')
    curr_df = bullpen_team_data_dict[visit_team]
    if date_dblhead in curr_df.index:
        for col in raw_cols_to_add:
            col_add_dict[col+'_v'][i] = curr_df.loc[date_dblhead,col]
    else:
        print(f'no match for {visit_team} date {date_dblhead}')

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000


In [12]:
for col in cols_to_add:
    df.reset_index(drop=True, inplace=True)
    df[col] = col_add_dict[col]

In [13]:
df.to_csv('df_bp7.csv', index=False)